In [1]:
from MFEA_lib.tasks.Benchmark.IDPC_EDU import IDPC_EDU_benchmark
from MFEA_lib.model import MFEA_base, MFEA_surrogate
from MFEA_lib.model.utils import *
from MFEA_lib.operators.Crossover import *
from MFEA_lib.operators.Mutation import *
from MFEA_lib.operators.Selection import *


In [2]:
tasks, IndClass = IDPC_EDU_benchmark.get_tasks(1)


baseModel = MFEA_surrogate.model()
baseModel.compile(
    IndClass= IndClass,
    tasks= tasks,
    # crossover= newSBX(nc = 2, gamma= 0.4, alpha= 6),
    crossover= IDPCEDU_Crossover(),
    mutation= IDPCEDU_Mutation(),
    selection= ElitismSelection()
)
solve = baseModel.fit(
    nb_generations = 1, rmp = 0.3, nb_inds_each_task= 1000, 
    bound_pop= [0, 1], evaluate_initial_skillFactor= True
)

Reading data...


100%|██████████| 1/1 [00:02<00:00,  2.52s/it]


Time: 00m 0.91s  100 % [====================>]  Cost: 1.30E+01  ,  
END!


# Create dataset

In [22]:
ls_inds = baseModel.populations[0].ls_subPop[0].ls_inds

In [21]:
tasks[0].__dict__.keys()

dict_keys(['file', 'datas', 'num_edges', 'dim', 'name', 'use_surrogate', 'num_nodes', 'num_domains', 'edges', 'source', 'target', 'count_paths'])

In [29]:
task = tasks[0]
source = task.source
target = task.target
count_paths = task.count_paths
edges = task.edges
num_nodes = task.num_nodes
num_edges = task.num_edges

In [30]:
import torch
import torch.nn as nn

In [33]:
src_trg = [0 for i in range(num_nodes)]
edge_index = []
edge_attribute = []

src_trg[source] = -1
src_trg[target] = 1

for i in range(count_paths.shape[0]):
    for j in range(count_paths.shape[1]):
        n = count_paths[i][j]

        edge_index.extend( [[i , j] for _ in range(n)] )
        edge_attribute.extend( [edges.get(f'{i}_{j}_{k}') for k in range(n)] )
        
src_trg = torch.tensor(src_trg, dtype = torch.float)
edge_index = torch.tensor(edge_index, dtype= torch.long).reshape(2, -1)
edge_attribute = torch.tensor(edge_attribute, dtype= torch.long)

In [110]:
sys.maxsize

9223372036854775807

In [39]:
datasets = []
labels = []
for ind in ls_inds:
    datasets.append(ind.genes)
    labels.append(ind.fcost)

# Model - GCNConv

In [163]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch_geometric.nn as gnn_nn
from torch.utils.data import Datasets   , DataLoaders

ImportError: cannot import name 'Datasets' from 'torch.utils.data' (c:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\utils\data\__init__.py)

In [64]:
src_trg = [0 for i in range(num_nodes)]
edge_index = []
edge_weight = []
edge_attribute = []

src_trg[source] = -1
src_trg[target] = 1

for i in range(count_paths.shape[0]):
    for j in range(count_paths.shape[1]):
        n = count_paths[i][j]
        if(n == 0):
            continue
        weight = [edges.get(f'{i}_{j}_{k}')[0] for k in range(n)]
        edge_index.append([i , j])
        edge_weight.append(sum(weight))
        #edge_attribute.extend( [edges.get(f'{i}_{j}_{k}') for k in range(n)] )
        
src_trg = torch.tensor(src_trg, dtype = torch.float)
edge_index = torch.tensor(edge_index, dtype= torch.long).t().contiguous()
edge_weight = torch.tensor(edge_weight, dtype = torch.float)
#edge_attribute = torch.tensor(edge_attribute, dtype= torch.long)

In [140]:
datasets = []
labels = []
MAX_INT = sys.maxsize
for i in range(len(ls_inds)):
    x_genes = torch.Tensor(ls_inds[i].genes)
    x_src_trg = src_trg.unsqueeze(0)
    _x = torch.cat((x_genes, x_src_trg)).permute(1,0)
    datasets.append(_x)
    labels.append(ls_inds[i].fcost if ls_inds[i].fcost < MAX_INT else 100)
    
labels = torch.Tensor(labels).unsqueeze(1)
#datasets

In [100]:
class GNN_GCN(nn.Module):
    def __init__(self, in_channels, hid_channels, num_nodes):
        super(GNN_GCN, self).__init__()
        self.gc1 = gnn_nn.GCNConv(in_channels, hid_channels)
        self.gc2 = gnn_nn.GCNConv(hid_channels, 1)
        self.fc = nn.Linear(num_nodes, 1)

    def forward(self, x, edge_index, edge_weight):
        x = F.relu(self.gc1(x, edge_index, edge_weight))
        x = self.gc2(x, edge_index, edge_weight)
        x = x.squeeze(1)
        x = self.fc(x)
        return x

In [148]:
labels[2]

tensor([13.])

In [192]:
class SurrogatePipeline():
    def __init__(self, input_dim, hidden_dim, num_nodes, learning_rate, device= None):
        self.device = device if device else 'cpu'
        self.model = GNN_GCN(input_dim, hidden_dim, num_nodes).to(self.device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=learning_rate)
        self.criteria = nn.RMSELoss()
        self.learning_rate = learning_rate
        
    def train(self, datasets, outputs, edge_index, edge_weight, epochs = 5):
        for epoch in range(epochs):
            losses = []
            for i in range(len(datasets)):
                x = datasets[i].to(self.device)
                edge_index = edge_index.to(self.device)
                edge_weight = edge_weight.to(self.device)

                output = outputs[i].to(self.device)
                pred = self.model(x, edge_index, edge_weight)
                loss = self.criteria(pred, output)
                losses.append(loss.item())
                # if i % 100 == 0:
                #     print(f'Loss: {loss}')
                
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
            print(f'Epoch {epoch} - Loss: {np.mean(losses)}')

    def predict(self, input):
        with torch.no_grad():
            input = torch.Tensor(input.flatten()).to(self.device)
            return self.model(input)

    def save_pipeline(self, path):
        pass

In [194]:
sr_pp = SurrogatePipeline(3, 10, num_nodes=10, learning_rate=4e-4, device='cuda')
sr_pp.train(datasets, labels, edge_index, edge_weight, epochs=100)

Epoch 0 - Loss: 107.2180230628816
Epoch 1 - Loss: 303.03579158806434
Epoch 2 - Loss: 311.2269177397946
Epoch 3 - Loss: 312.65274710823735
Epoch 4 - Loss: 312.4519780803384
Epoch 5 - Loss: 311.8659458040227
Epoch 6 - Loss: 311.4074625868327
Epoch 7 - Loss: 311.4145655324133
Epoch 8 - Loss: 312.0714562056883
Epoch 9 - Loss: 313.87060741694324
Epoch 10 - Loss: 317.957814127397
Epoch 11 - Loss: 324.58463878014453
Epoch 12 - Loss: 329.19024681980534
Epoch 13 - Loss: 331.70607549006587
Epoch 14 - Loss: 333.64689503167153


KeyboardInterrupt: 

In [181]:
gnn_gcn = GNN_GCN(3,10,10)
x123 = gnn_gcn(_x, edge_index, edge_weight)

In [183]:
criteria = nn.MSELoss()
criteria(labels[0], labels[44]).item()

16.0